# SQL Package

Provides simple functionality to interact with a PostgreSQL server using Python classes.

**Overview of functionality:**
* Database(self, user, password, host, dbname, port)
    * properties
        * user
        * password
        * host
        * dbname
        * port
    * methods
        * create(name) x
        * connect()
        * drop(name)
* Table(self, dbname, table, schema)
    * accepts db properties
    * properties
        * connect() --> inherited
        * fetch_data(sql, con, parse_dates)
        * get_names()
        * format_names(char_dict)
        * update_names(names_dict)
        * add_columns(columns_list, type=None)
        * compare_column_order(dataframe)
        * match_columns(dataframe)
        * save_csv(data, local_path, match_column_order=True)
        * update_values(local_path, container_path)
        * update_types(types_dict)
        * close()

## Setup

In [1]:
import os
import sys
from pathlib import Path
#sys.path[0] = str(Path(__file__).resolve().parents[2]) # Set path for custom modules
import warnings
from io import StringIO

# Set path for modules
sys.path[0] = '../'

from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

# SQL libraries
import psycopg2

# Set notebook display options
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Get project root directory
root_dir = os.path.dirname(os.getcwd())

In [2]:
#sys.modules

In [3]:
class Database():
    
    # if modulename not in sys.modules: print...
    load_dotenv(find_dotenv());
    
    def __init__(self, user=None, password=None,
                 dbname=None, host=None, port=None):
        
        # Loaded from .env if not explicit
        self.user = user if user is not None else os.getenv("POSTGRES_USER")
        self.password = password if password is not None else os.getenv("POSTGRES_PASSWORD")
        self.dbname = dbname if dbname is not None else os.getenv("POSTGRES_DB")
        self.host = host if host is not None else os.getenv("DB_HOST")
        self.port = port if port is not None else os.getenv("DB_PORT")
        
        
        # Root directory
        self._root_dir = os.path.dirname(os.getcwd())
        #sys.path[0] = str(Path(__file__).resolve().parents[2])
        
    def _connect(self):

        """
        Connects to PostgreSQL database using psycopg2 driver. Same
        arguments as psycopg2.connect().

        Params
        --------
        dbname
        user
        password
        host
        port
        connect_timeout
        """

        try:
            con = psycopg2.connect(dbname=self.dbname,
                                   user=self.user,
                                   password=self.password,
                                    host=self.host, 
                                    port=self.port,
                                  connect_timeout=3)            

        except Exception as e:
            print('Error:\n', e)
            return None


        return con
    
    @property
    def _con(self):
        try:
            con = self._connect()
            print('Connected as user "{}" to database "{}" on http://{}:{}.'.format(self.user,self.dbname,
                                                               self.host,self.port))
            con.close()
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()
        

In [4]:
db = Database()

In [344]:
class Table(Database):
    def __init__(self, user=None, password=None, dbname=None, host=None, port=None, table=None):
        super().__init__(user, password, dbname, host, port)
        
        self.table = table
        
        # Loaded from .env if not explicit
        self.user = user if user is not None else os.getenv("POSTGRES_USER")
        self.password = password if password is not None else os.getenv("POSTGRES_PASSWORD")
        self.dbname = dbname if dbname is not None else os.getenv("POSTGRES_DB")
        self.host = host if host is not None else os.getenv("DB_HOST")
        self.port = port if port is not None else os.getenv("DB_PORT")
    
    # Connect to database
    def __connect(self):
        return super(Table, self)._connect()
    
    # Check info on connection
    def __con(self):
        return super(Table, self)._con
    
    # Fetch data from sql query
    def fetch_data(self, sql, coerce_float=False, parse_dates=None):
        
        con = self.__connect()
        
        # Fetch fresh data
        data = pd.read_sql_query(sql=sql, con=con, coerce_float=coerce_float, parse_dates=parse_dates)

        # Replace None with np.nan
        data.fillna(np.nan, inplace=True)
        
        # Close db connection
        con.close()

        return data
    
    # Get names of column
    def get_names(self):
        
        # Specific query to retrieve table names
        sql = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'{}'".format(self.table)
        
        # Run query and extract
        con = self.__connect()
        data = pd.read_sql_query(sql, con)
        column_series = data['column_name']
        con.close()
    
        return column_series

    # Standardize column names using dictionary of character replacements
    def reformat_names(self, replace_map):
        
        series = self.get_names()
        
        def replace_chars(text):
            for oldchar, newchar in replace_map.items():
                text = text.replace(oldchar, newchar).lower()
            return text
        
        return series.apply(replace_chars)
    
    # Update column names in db table
    def update_names(self, replace_map):
        
        # Extract current columns in table
        old_columns = self.get_names()
        
        # Create list of reformatted columns to replace old columns 
        new_columns = self.reformat_names(replace_map)
    
        # SQL query string to change column names
        sql = 'ALTER TABLE {} '.format(self.table) + 'RENAME "{old_name}" to {new_name};'

        sql_query = []

        # Iterate through old column names and replace each with reformatted name 
        for idx, name in old_columns.iteritems():
            sql_query.append(sql.format(old_name=name, new_name=new_columns[idx]))
            
        # Join list to string
        sql_query = '\n'.join(sql_query)
        
        # Execute query against database
        con = self.__connect()
        try:
            print("Connecting...")
            cur = con.cursor()
            print('Executing query on table "{}"...'.format(self.table))
            cur.execute(sql_query)
            con.commit()
            cur.close()
            print("Table names are updated.")
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()
                
    # Add new columns to database
    def add_columns(self, data):

        # Get names of current columns in PostgreSQL table
        current_names = self.get_names()

        # Get names of updated table not in current table
        updated_names = data.columns.tolist()
        new_names = list(set(updated_names) - set(current_names))

        # Check names list is not empty
        if not new_names:
            print("Table columns are already up to date.")
            return

        # Format strings for query
        alter_table_sql = "ALTER TABLE {db_table}\n"
        add_column_sql = "\tADD COLUMN {column} TEXT,\n"

        # Create a list and append ADD column statements
        sql_query = [alter_table_sql.format(db_table=self.table)]
        for name in new_names:
            sql_query.append(add_column_sql.format(column=name))

        # Join into one string
        sql_query = ''.join(sql_query)[:-2] + ";"

        # Execute query against database
        con = self.__connect()
        try:
            print("Connecting...")
            cur = con.cursor()
            print('Executing query on table "{}"'.format(self.table))
            cur.execute(sql_query)
            con.commit()
            cur.close()
            print("Columns added successfully.")
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()

    # Compare order of columns in dataframe against order of columns in database                
    def compare_column_order(self, data):
        
        # Get columns from database as list
        db_columns = self.get_names().tolist()
        
        # Select columns from dataframe as list
        data_columns = data.columns.tolist()
        
        if set(data_columns) == set(db_columns):
            
            str1 = 'Dataframe columns match table "{}" '.format(self.table)
            
            if data_columns == db_columns:
                print(str1 + "and are in identical order.")
                return True
            else:
                print("but are not in identical order.")                
                return False            
        else:
            if len(data_columns) > len(db_columns):
                print('Dataframe has more columns than table "{}".'.format(self.table))
                return False
            else:
                print('Dataframe has less columns than table "{}".'.format(self.table))
                return False
    
    # Rearrange the order of columns in dataframe to match order in table
    def match_column_order(self, data):
        
        # Get columns from database as list
        db_columns = self.get_names().tolist()

        # Select columns from dataframe as list
        data_columns = data.columns.tolist()
        
        if set(data_columns) == set(db_columns):
            if data_columns != db_columns:
                print('Rearranged dataframe columns to match table "{}".'.format(self.table))
                return data[db_columns]
            else:
                print('Dataframe columns already match table "{}".'.format(self.table))
                return data
        else:
            if len(data_columns) > len(db_columns):
                print('Dataframe has more columns than table "{}".'.format(self.table))
                return data
            else:
                print('Dataframe has less columns than table "{}".'.format(self.table))
                return data
            
    ##### RESUME HERE
    # Builds a query to update postgres from a csv file
    def update_values(self, data, id_col, types_dict, columns=None, sep=','):
        
        # Inherit create_table method and create a temp_table
        
        # CREATE TABLE query
        tmp_table = "tmp_" + self.table

        column_names = self.get_names().tolist() if not columns else columns
        names = ',\n\t'.join(['{}'.format(name) + " TEXT" for name in column_names])
        
        sql_create_tmp_table = 'DROP TABLE IF EXISTS {};\n\n'.format(tmp_table)
        sql_create_tmp_table = sql_create_tmp_table + 'CREATE TABLE {tmp_table} (\n\t{names}\n);\n\n' \
                                .format(tmp_table=tmp_table, names=names)
        
        # Append types to temp table
        
        
        # UPDATE query
        sql_update_query = 'UPDATE {db_table}\n'.format(db_table=self.table)
        
        # SET statements
        sql_set = ["SET "]
        for name in column_names:
            set_sql = "{name} = {tmp_name},\n\t".format(name=name, tmp_name=tmp_table + '.' + name)
            sql_set.append(set_sql)
        sql_set = ''.join(sql_set)
        sql_set = sql_set[:-3] + "\n"
        
        
        # FROM clause
        sql_from = "FROM {tmp_table}\nWHERE {db_table}.{id_col} = {tmp_table}.{id_col};\n\n" \
                            .format(tmp_table=tmp_table, db_table=self.table, id_col=id_col)
        
        # DROP tmp_table
        sql_drop = 'DROP TABLE {};\n'.format(tmp_table)
        
        # Concatenate queries
        sql_query_1 = sql_create_tmp_table
        sql_query_2 = sql_update_query + sql_set + sql_from + sql_drop


        print(sql_query_1)
        print("RUN COPY FROM TEMP TABLE")
        print(sql_query_2)
        
        # Run update query
        data_buffer = StringIO(data.to_csv(header=False, index=False))
        con = self.__connect()
        try:
            print("Connecting...")
            cur = con.cursor()
            print('Executing query on table "{}"'.format(self.table))
            
            # Create tmp_table
            cur.execute(sql_query_1)
            
            # Copy into temp_table
            data_buffer.read()
            cur.copy_from(file=data_buffer, table=tmp_table, columns=columns, sep=sep)
            data_buffer.close()
            
            # Update from temp_table into table and delete temp
            cur.execute(sql_query_2)
            con.commit()
            cur.close()
            print("Database updated successfully.")
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()

In [345]:
# Save csv with option to match order of columns in postgres
def save_csv(data, path, index=False):

    # Check unique columns
    if data.columns.tolist() != data.columns.unique().tolist():
        raise IndexError("Dataframe has duplicate columns.")

    if index:
        warnings.warn('Setting "index=True" may cause problems when importing from csv file.')


    # Write to csv
    data.to_csv(path, index=False)

In [346]:
permits = Table(table="permits_raw")
data = permits.fetch_data(sql="SELECT * FROM permits_raw;")

In [340]:
permits._con

Connected as user "postgres" to database "permits" on http://localhost:5432.


In [341]:
data.head()

,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code,longitude,full_address,latitude
0,2027,4,011,TR 26618,NaN,11,NaN,17042-90000-21398,Issued,2017-08-30,Plumbing,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,INTERNET,2017-08-30,23311,NaN,23311,NaN,W,WINDOM,ST,NaN,NaN,NaN,91304,NaN,NaN,NaN,NaN,NaN,NaN,OWNER-BUILDER,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,KRISTEN,IMHOFF,NaN,5048 CAMPO,NaN,"WOODLAND HILLS, CA",RE11-1,NaN,0.0,1344.22,12,"(34.20709, -118.63795)",Owner-Bldr,NaN,NaN,-118.63795,23311 W WINDOM ST 91304,34.20709
1,4317,3,***,TR 30210-C,NaN,LT 1,NaN,15044-90000-08405,Permit Finaled,2015-09-10,HVAC,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,INTERNET,2015-08-18,1823,1/2,1823,1/2,S,THAYER,AVE,NaN,NaN,NaN,90025,NaN,NaN,NaN,NaN,NaN,NaN,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,2016-06-30,BRETT,HOFFER,NaN,18650 PARTHENIA ST,NaN,"NORTHRIDGE, CA",R3-1-O,NaN,0.0,2671.0,5,"(34.05474, -118.42628)",Net Applicant,NaN,NaN,-118.42628,1823 S THAYER AVE 90025,34.05474
2,5005,10,017,CHESTERFIELD SQUARE,NaN,465,16SL57806,16016-70000-02464,Permit Finaled,2017-08-01,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,SOUTH LA,2016-02-04,2122,NaN,2122,NaN,W,54TH,ST,NaN,NaN,NaN,90062,General rehabilitation for single family dwell...,40000.0,NaN,NaN,NaN,NaN,OWNER-BUILDER,NaN,NaN,NaN,NaN,0,JAVIER,NaN,TALAMANTES,NaN,JAVIER,TALAMANTES,OWNER-BUILDER,NaN,NaN,NaN,C2-1VL,NaN,NaN,2325.0,8,"(33.99307, -118.31668)",Owner-Bldr,1.0,NaN,-118.31668,2122 W 54TH ST 90062,33.99307
3,5154,23,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,2014-08-13,Bldg-Alter/Repair,Apartment,Plan Check,NaN,NaN,VAN NUYS,2014-08-13,415,NaN,415,NaN,S,BURLINGTON,AVE,NaN,1-30,1-30,90057,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.0,NaN,NaN,NaN,NaN,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,2015-11-30,LINDA,MARTON,NaN,710 WILSHIRE BLVD,NaN,"SANTA MONICA, CA",R4-1,NaN,NaN,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5.0,NaN,-118.26997,415 S BURLINGTON AVE 90057,34.06012
4,4404,30,010,TR 12086,NaN,2,NaN,16044-30000-09658,Permit Finaled,2016-08-29,HVAC,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,WEST LA,2016-08-22,315,NaN,315,NaN,S,OCEANO,DR,NaN,NaN,NaN,90049,NaN,NaN,NaN,NaN,NaN,NaN,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,2018-07-31,NaN,NaN,NaN,NaN,NaN,NaN,RS-1,NaN,0.0,2640.0,11,"(34.05707, -118.4732)",Contractor,NaN,NaN,-118.4732,315 S OCEANO DR 90049,34.05707


In [342]:
permits.get_names()[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object

In [260]:
# Map of character replacements
replace_map = {' ': '_', '-': '_', '#': 'No', '/': '_', 
               '.': '', '(': '', ')': '', "'": ''}

permits.reformat_names(replace_map)[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object

In [261]:
permits.update_names(replace_map)

Connecting...
Executing query on table "permits_raw"...
Error:
 column "assessor_book" of relation "permits_raw" already exists



In [262]:
permits.add_columns(data)

Table columns are already up to date.


In [263]:
permits.get_names()[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object

In [264]:
permits.compare_column_order(data)

Dataframe columns match table "permits_raw" and are in identical order.


True

In [265]:
permits.match_column_order(data).head()

Dataframe columns already match table "permits_raw".


,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code,longitude,full_address,latitude
0,2027,4,011,TR 26618,NaN,11,NaN,17042-90000-21398,Issued,2017-08-30,Plumbing,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,INTERNET,2017-08-30,23311,NaN,23311,NaN,W,WINDOM,ST,NaN,NaN,NaN,91304,NaN,NaN,NaN,NaN,NaN,NaN,OWNER-BUILDER,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,KRISTEN,IMHOFF,NaN,5048 CAMPO,NaN,"WOODLAND HILLS, CA",RE11-1,NaN,0.0,1344.22,12,"(34.20709, -118.63795)",Owner-Bldr,NaN,NaN,-118.63795,23311 W WINDOM ST 91304,34.20709
1,4317,3,***,TR 30210-C,NaN,LT 1,NaN,15044-90000-08405,Permit Finaled,2015-09-10,HVAC,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,INTERNET,2015-08-18,1823,1/2,1823,1/2,S,THAYER,AVE,NaN,NaN,NaN,90025,NaN,NaN,NaN,NaN,NaN,NaN,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,2016-06-30,BRETT,HOFFER,NaN,18650 PARTHENIA ST,NaN,"NORTHRIDGE, CA",R3-1-O,NaN,0.0,2671.0,5,"(34.05474, -118.42628)",Net Applicant,NaN,NaN,-118.42628,1823 S THAYER AVE 90025,34.05474
2,5005,10,017,CHESTERFIELD SQUARE,NaN,465,16SL57806,16016-70000-02464,Permit Finaled,2017-08-01,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,SOUTH LA,2016-02-04,2122,NaN,2122,NaN,W,54TH,ST,NaN,NaN,NaN,90062,General rehabilitation for single family dwell...,40000.0,NaN,NaN,NaN,NaN,OWNER-BUILDER,NaN,NaN,NaN,NaN,0,JAVIER,NaN,TALAMANTES,NaN,JAVIER,TALAMANTES,OWNER-BUILDER,NaN,NaN,NaN,C2-1VL,NaN,NaN,2325.0,8,"(33.99307, -118.31668)",Owner-Bldr,1.0,NaN,-118.31668,2122 W 54TH ST 90062,33.99307
3,5154,23,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,2014-08-13,Bldg-Alter/Repair,Apartment,Plan Check,NaN,NaN,VAN NUYS,2014-08-13,415,NaN,415,NaN,S,BURLINGTON,AVE,NaN,1-30,1-30,90057,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.0,NaN,NaN,NaN,NaN,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,2015-11-30,LINDA,MARTON,NaN,710 WILSHIRE BLVD,NaN,"SANTA MONICA, CA",R4-1,NaN,NaN,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5.0,NaN,-118.26997,415 S BURLINGTON AVE 90057,34.06012
4,4404,30,010,TR 12086,NaN,2,NaN,16044-30000-09658,Permit Finaled,2016-08-29,HVAC,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,WEST LA,2016-08-22,315,NaN,315,NaN,S,OCEANO,DR,NaN,NaN,NaN,90049,NaN,NaN,NaN,NaN,NaN,NaN,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,2018-07-31,NaN,NaN,NaN,NaN,NaN,NaN,RS-1,NaN,0.0,2640.0,11,"(34.05707, -118.4732)",Contractor,NaN,NaN,-118.4732,315 S OCEANO DR 90049,34.05707


In [266]:
path = root_dir + "/data/interim/test.csv"

save_csv(data, path, index=True)

/Users/gregory/anaconda3/envs/permits-data-env/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Setting "index=True" may cause problems when importing from csv file.
  if __name__ == '__main__':


In [347]:
permits.update_values(data, id_col='pcis_permit_no')

DROP TABLE IF EXISTS tmp_permits_raw;

CREATE TABLE tmp_permits_raw (
	assessor_book TEXT,
	assessor_page TEXT,
	assessor_parcel TEXT,
	tract TEXT,
	block TEXT,
	lot TEXT,
	reference_no_old_permit_no TEXT,
	pcis_permit_no TEXT,
	status TEXT,
	status_date TEXT,
	permit_type TEXT,
	permit_sub_type TEXT,
	permit_category TEXT,
	project_number TEXT,
	event_code TEXT,
	initiating_office TEXT,
	issue_date TEXT,
	address_start TEXT,
	address_fraction_start TEXT,
	address_end TEXT,
	address_fraction_end TEXT,
	street_direction TEXT,
	street_name TEXT,
	street_suffix TEXT,
	suffix_direction TEXT,
	unit_range_start TEXT,
	unit_range_end TEXT,
	zip_code TEXT,
	work_description TEXT,
	valuation TEXT,
	floor_area_la_zoning_code_definition TEXT,
	no_of_residential_dwelling_units TEXT,
	no_of_accessory_dwelling_units TEXT,
	no_of_stories TEXT,
	contractors_business_name TEXT,
	contractor_address TEXT,
	contractor_city TEXT,
	contractor_state TEXT,
	license_type TEXT,
	license_no TEXT,
	principal_firs

In [189]:
permits_fetch_

62